# Quantum Ansatz Optimisation Algorithm for the Quadratic Assignment Problem

In [3]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# importing Qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [4]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [5]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

matrixMade = ['./data/made4.csv', 
              './data/made5.csv', 
              './data/made6.csv', 
              './data/made7.csv', 
              './data/made8.csv', 
              './data/made9.csv']


#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = './data/tai10a.csv'
matrix_size_11 = './data/made11.csv'
matrix_size_12 = './data/tai12a.csv'
matrix_size_15 = './data/chr15a.csv' 
matrix_size_20 = './data/chr20a.csv'
matrix_size_26 = './data/bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = './data/tai40a.csv'
matrix_size_60 = './data/tai60.csv'
matrix_size_80 = './data/tai80.csv'
matrix_size_256 = './data/tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

Below is the fitness function for the QAP using the  Quadratic 0–1 formulation

In [6]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Below, IBMQ account is loaded and the backends displayed

In [7]:
from qiskit import IBMQ

 # Load account from disk

IBMQ.save_account("3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810", overwrite = True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>]

In [8]:
provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('

These functions are needed to find and convert the answer

In [9]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [10]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    array of [feasible binary 1D numpyarray,probability of this answer]
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    final = np.zeros(2)
    
    while counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        counter += 1
        if feasible ==True:
            
            frequency = bestinarray[counter][1]
            final = np.vstack((final,[bestasint[1:], frequency/total]))
            feasible = False
    return final[1:]

This function outputs the QAOA solutions

In [11]:
def testing_quantum(machine, ins,SPtrial,SHOT):
    """
    Input:
    machine: the name of available machine
    ins: QAP instances
    
    Output:
    outmatrix : [time, eigensolution value,machine value,instance]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #Eigensolver
    ee = NumPyEigensolver(qubitOp_docplex, k=1)#potential error
    result1 = ee.run() #potential error
    print('Eigen energy:', result1['eigenvalues'])
    print('Eigen QAP objective:', np.real(result1['eigenvalues'])[0] + offset_docplex)

    #QAOA
    seed = 10598
    spsa = SPSA(max_trials=SPtrial)
    backend = Aer.get_backend(machine)
    problem = QuadraticProgram(qubitOp_docplex)
    #backend = provider.get_backend(j)
    
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation = False, shots = SHOT)
    qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=5, quantum_instance=quantum_instance)
    result = qaoa.run(backend)
    print('QAOA energy:', result['eigenvalue'])
    print('QAOA time:', result['optimizer_time'])
    x = choose_best_feasible(result['eigenstate'])
    soln = []
    if x[0]==0:
        print("No feasible solution found")
        return [[],np.array([[[],[],np.inf]])]
    else:
        for i in x:
            soln.append(qap_value(i[0],MatrixFlow,MatrixLoc))
        print('QAOA QAP objective:', soln)
        nsoln = len(soln)
        return machine,np.hstack((x,np.array(soln).reshape(nsoln,1))),result['optimizer_time']
    

In [12]:
#Uncomment to run on the machines

#machines = ['ibmq_qasm_simulator', 'ibmqx2','ibmq_16_melbourne', 'ibmq_vigo', 'ibmq_ourense', 'ibmq_valencia', 
#'ibmq_london', 'ibmq_burlington', 'ibmq_johannesburg', 'ibmq_rochester', 'ibmq_cambridge', 'ibmq_paris']
#instances = ['made4.csv','made4.csv','made5.csv','made6.csv','made7.csv','made8.csv']

#eigens = testing_quantum(machines, instances)

In [13]:
machines = 'qasm_simulator'
instances = matrix_size_4

ans = testing_quantum(machines, instances, 300, 1000)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

KeyboardInterrupt: 

In [ ]:
def gridsearch(machine, ins, trials, shots):
    final = np.array([])
    finalind = np.array([])
    for t in trials:
        for s in shots:
            final = np.append(final,[testing_quantum(machine, ins, t, s)[1][:,2][0],t,s])
            print(t,s)
    
    return final
    

In [12]:
machines = 'qasm_simulator'
instances = matrix_size_4

gridsearch(machines, instances, [100,200,300,400], [100,500,1000,1500])

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (354.65451276977063+0j)
QAOA time: 243.20709896087646
No feasible solution found
200 1500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (-244.07780556359725+0j)
QAOA time: 487.452543258667
QAOA QAP objective: [1096]
400 100
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x

array([  inf,  100.,  100.,   inf,  100.,  500.,  990.,  100., 1000.,
         inf,  100., 1500.,   inf,  200.,  100.,   inf,  200.,  500.,
       1052.,  200., 1000.,   inf,  200., 1500.,   inf,  300.,  100.,
         inf,  300.,  500.,   inf,  300., 1000.,   inf,  300., 1500.,
       1096.,  400.,  100., 1096.,  400.,  500.,  838.,  400., 1000.,
         inf,  400., 1500.])

In [14]:
def thirtytrials(machine, ins, t, s):
    final = np.zeros(2)
    acount95 = 0
    acount99 = 0
    mcount95 = 0
    mcount99 = 0
    for i in range(30):
        ans = testing_quantum(machine, ins, t, s)[1][:,2]
        minimum = np.min(ans)
        final = np.vstack((final,[ans[0],minimum]))
        #SR95
        if ans[0] <= 1.05*790:
            acount95+=1
        if minimum <= 1.05*790:
            mcount95+=1
        #sr99
        if ans[0] <= 1.01*790:
            acount99+=1
        if minimum <= 1.01*790:
            mcount99+=1
    return 100*acount95/30, 100*acount99/30, 100*mcount95/30, 100*mcount99/30

In [15]:
t = 400
s = 1000
machines = 'qasm_simulator'
instances = matrix_size_4

thirtytrials(machines, instances, t, s)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


QAOA QAP objective: [990]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (-112.333465174001+0j)
QAOA time: 443.2580027580261
No feasible solution found
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (-200.46969164392783+0j)
QAOA time: 666.5979588031769
QAOA QAP objective: [838]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (501.1946891640157+0j)
QAOA time: 487.5273656845093
No feasible solution found
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (-407.93714872892906+0j)
QAOA time: 469.7547798156738
No feasible solution found
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
QAOA energy: (984.8082377297662+0j)
QAOA time: 447.38091230392456
QAOA QAP objective: [1008]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3

(10.0, 10.0, 13.333333333333334, 10.0)

The result (10.0, 10.0, 13.333333333333334, 10.0) implies that the most probable output solution is 95% from the minimum 10% of the time and 99% from the minimum 10% of the time while the overall minimum feasible output is 95% within the solution 13.33% of the time and is 99% within the solution 10% of the time.

In [16]:
(452.0876495838165+443.96349906921387+452.3047547340393+462.7005636692047+463.76987075805664+448.273300409317+440.90419459342957+443.2580027580261+439.41539311408997+446.9959924221039+510.1479296684265+649.4923319816589+666.5979588031769+624.9015824794769+647.3900508880615+566.737596988678+480.9131381511688+487.5273656845093+469.71762108802795+514.3556642532349+471.7129228115082+477.521865606308+469.7547798156738+481.4351532459259+ 454.8790397644043+446.4799418449402+439.74220991134644+447.38091230392456+441.2286374568939+451.3744525909424)/30

489.76547921498616